# 무역 데이터 EDA (HS4 / item_id 단위 EDA 및 micro-level deep dive)

In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns


df_train = pd.read_csv('train.csv')
print(df_train.head(20))



     item_id  year  month  seq  type   hs4      weight  quantity       value
0   DEWLVASR  2022      1  1.0     1  3038     14858.0       0.0     32688.0
1   ELQGMQWE  2022      1  1.0     1  2002     62195.0       0.0    110617.0
2   AHMDUILJ  2022      1  1.0     1  2102     18426.0       0.0     72766.0
3   XIPPENFQ  2022      1  1.0     1  2501     20426.0       0.0     11172.0
4   FTSVTTSR  2022      1  1.0     1  2529    248000.0       0.0    143004.0
5   XMKRPGLB  2022      1  1.0     1  2529    558000.0       0.0    205676.0
6   RCBZUSIM  2022      1  1.0     1  2805         0.0       0.0       459.0
7   SUOYXCHP  2022      1  1.0     1  2805         0.0       0.0       482.0
8   ZKENOUDA  2022      1  1.0     1  2805         2.0       0.0      2783.0
9   WQMVCOEM  2022      1  1.0     1  2805       187.0       0.0    217777.0
10  BSRMSVTC  2022      1  1.0     1  2805       111.0       0.0     33952.0
11  DDEXPPXU  2022      1  1.0     1  2807       387.0       0.0     13158.0

## 1. 전처리

- zero / missing flag 생성:
  - value, weight, quantity의 0은 정상적일 수 있음 👉 제거 X
  - 대신 zero 패턴 자체를 feature로 인식하기 위해 flag 생성.

- 결측치 처리:
  - weight, quantity, value 결측은 보간하지 않음 (의미 있는 결측이기 때문)

- item_id / HS4 단위 monthly pivot 생성
  - → Comovement 분석의 기본 단위로 사용될 시계열 구조 확보.

In [3]:
# Zero flag 생성
df_train['value_zero'] = (df_train['value'] == 0).astype(int)
df_train['weight_zero'] = (df_train['weight'] == 0).astype(int)
df_train['quantity_zero'] = (df_train['quantity'] == 0).astype(int)

# Missing flag 생성
df_train['value_missing'] = df_train['value'].isna().astype(int)
df_train['weight_missing'] = df_train['weight'].isna().astype(int)
df_train['quantity_missing'] = df_train['quantity'].isna().astype(int)

In [4]:
hs2_map = {
    "01": "산동물",
    "02": "육과 식용설육",
    "03": "어패류",
    "04": "낙농품·조란·천연동물성 식품",
    "05": "기타 동물성 생산품",
    "06": "산수목·꽃",
    "07": "채소",
    "08": "과실·견과류",
    "09": "커피·향신료",

    "10": "곡물",
    "11": "밀가루·전분",
    "12": "채유용 종자·인삼",
    "13": "식물성 엑스",
    "14": "기타 식물성 생산품",
    "15": "동식물성 유지",
    "16": "육·어류 조제품",
    "17": "당류·설탕과자",
    "18": "코코아·초콜릿",
    "19": "곡물·곡분 주제품·빵류",

    "20": "채소·과실의 조제품",
    "21": "기타 조제식료품",
    "22": "음료·주류·식초",
    "23": "조제사료",
    "24": "담배",
    "25": "토석류·소금",
    "26": "광·슬랙·회",
    "27": "광물성 연료·에너지",
    "28": "무기화합물",
    "29": "유기화합물",

    "30": "의료용품",
    "31": "비료",
    "32": "염료·안료·페인트·잉크",
    "33": "향료·화장품",
    "34": "비누·계면활성제·왁스",
    "35": "카세인·알부민·변성전분·효소",
    "36": "화약류·성냥",
    "37": "필름·인화지·사진용 재료",
    "38": "기타 화학공업 품목",
    "39": "플라스틱과 그 제품",

    "40": "고무와 그 제품",
    "41": "원피·가죽",
    "42": "가죽제품",
    "43": "모피·모피제품",
    "44": "목재·목탄",
    "45": "코르크·코르크 제품",
    "46": "조물재료 제품",
    "47": "펄프",
    "48": "지·판지",
    "49": "서적·신문·인쇄물",

    "50": "견·견사·견직물",
    "51": "양모·수모",
    "52": "면·면사·면직물",
    "53": "마류사·직물",
    "54": "인조 필라멘트 섬유",
    "55": "인조 스테이플 섬유",
    "56": "워딩·부직포",
    "57": "양탄자",
    "58": "특수 직물",
    "59": "침투·도포 직물",

    "60": "편물",
    "61": "의류(편물제)",
    "62": "의류(편물제 외)",
    "63": "기타 섬유제품·넝마",
    "64": "신발류",
    "65": "모자류",
    "66": "우산·지팡이",
    "67": "조제 우모·인조제품",
    "68": "석·시멘트·석면 제품",
    "69": "도자 제품",

    "70": "유리",
    "71": "귀석·반귀석·귀금속",
    "72": "철강",
    "73": "철강제품",
    "74": "동과 그 제품",
    "75": "니켈과 그 제품",
    "76": "알루미늄과 그 제품",
    "78": "연과 그 제품",
    "79": "아연과 그 제품",

    "80": "주석과 그 제품",
    "81": "기타 비금속",
    "82": "비금속 공구·스푼·포크",
    "83": "각종 비금속 제품",
    "84": "보일러·기계류",
    "85": "전기기기·TV·VTR",
    "86": "철도차량",
    "87": "일반차량",
    "88": "항공기",
    "89": "선박",

    "90": "광학·의료·측정·정밀기기",
    "91": "시계",
    "92": "악기",
    "93": "무기",
    "94": "가구·조명기구",
    "95": "완구·운동용품",
    "96": "잡품",
    "97": "예술품·골동품"
}


### 모든 item_id × 모든 month 패널 완성(Panel Completion)

현재 데이터는 모든 item_id가 모든 월에 등장하지 않음 → 공행성 분석에서 큰 문제됨.
동시에 존재하는 시계열끼리 비교해야 함

예:
- A item은 2022~2025 전 기간 존재
- B item은 특정 계절만 존재
- C item은 중간에 구멍 있음

이런 상태에서 comovement correlation 계산하면 시계열 길이가 불일치 → 오류 / 왜곡 발생, 상관계수 계산 불가<br>
- A : 2022.01 ~ 2025.07 모든 달에 존재 = 42개월
- C : 2022.03 ~ 2023.05 일부 달에 존재 = 15개월
→ 반드시 해야 하는 전처리:<br>
모든 item_id × 모든 month 조합을 만들고 없는 row는 0 또는 NaN으로 채우기

✅ 1단계: df_train에 date 컬럼 붙이기

아래 셀을 통째로 새로 실행해 줘:

✅ 2단계: panel(모든 item_id × 모든 month) 만들기

이제 date가 생겼다고 가정하고, 아래 셀 실행 👇

In [6]:
# ------------------------------------
# 0) date 컬럼 만들기 (year, month 기준)
# ------------------------------------
# year, month가 있고 date가 아직 없을 때만 생성
if "date" not in df_train.columns:
    df_train["date"] = pd.to_datetime(
        df_train["year"].astype(int).astype(str) + "-" +
        df_train["month"].astype(int).astype(str) + "-01"
    )

# 혹시 date가 object/string인 경우 datetime으로 변환
df_train["date"] = pd.to_datetime(df_train["date"])

# ------------------------------------
# 0-1) 거래 여부 플래그 (row 단위)
# ------------------------------------
# 이 row는 실제로 거래 기록이 있다 = 1
df_train["has_tx"] = 1

# ------------------------------------
# 1) 월별 집계 (중복거래 제거 핵심)
#    item_id + hs4 + date 기준으로 value/weight/quantity 합산
#    + zero / missing / 거래여부 플래그도 같이 올리기
# ------------------------------------
agg_cols = ["value", "weight", "quantity"]

# 숫자 컬럼은 "모두 결측이면 NaN 유지" 되도록 sum(min_count=1) 사용
agg_dict = {
    "value":    lambda x: x.sum(min_count=1),
    "weight":   lambda x: x.sum(min_count=1),
    "quantity": lambda x: x.sum(min_count=1),
    
    # zero / missing 플래그는 한 번이라도 1이면 1
    "value_zero":    "max",
    "weight_zero":   "max",
    "quantity_zero": "max",

    "value_missing":    "max",
    "weight_missing":   "max",
    "quantity_missing": "max",

    # 거래 있었는지 여부 (해당 월에 row가 1개 이상 있으면 합 > 0)
    "has_tx": "sum",
}

df_train_monthly = (
    df_train
    .groupby(["item_id", "hs4", "date"], as_index=False)
    .agg(agg_dict)
)

# has_tx를 0/1로 정리
df_train_monthly["has_tx"] = (df_train_monthly["has_tx"] > 0).astype(int)

print("월별 집계 결과 예시:")
print(df_train_monthly.head())

# ------------------------------------
# 2) 전체 month 범위 & item_id 목록 만들기
# ------------------------------------
min_month = df_train_monthly["date"].min()
max_month = df_train_monthly["date"].max()

full_months = pd.date_range(min_month, max_month, freq="MS")
all_items = df_train_monthly["item_id"].unique()

# ------------------------------------
# 3) item_id × date 전체 조합 (풀 패널 인덱스)
# ------------------------------------
panel = pd.MultiIndex.from_product(
    [all_items, full_months],
    names=["item_id", "date"]
).to_frame(index=False)

# ------------------------------------
# 4) 집계된 df_train_monthly와 merge해서 패널 완성
# ------------------------------------
df_panel = (
    panel.merge(df_train_monthly, on=["item_id", "date"], how="left")
         .sort_values(["item_id", "date"])
         .reset_index(drop=True)
)

# merge 이후: 거래 없던 month(has_tx가 NaN) → 0으로, 거래 있는 달은 1 유지
df_panel["has_tx"] = df_panel["has_tx"].fillna(0).astype(int)

# -----------------------------
# ⚠️ 핵심: 숫자 칼럼 NaN을 전부 0으로 채우지 않는다!
#    → "진짜 거래 없는 달"만 0으로 세팅
# -----------------------------
no_tx_mask = df_panel["has_tx"] == 0

# 거래 자체가 없던 달은 value/weight/quantity를 0으로
df_panel.loc[no_tx_mask, agg_cols] = 0

# 플래그들도 거래 없는 달은 0으로 세팅 (원하는 경우)
flag_cols = [
    "value_zero", "weight_zero", "quantity_zero",
    "value_missing", "weight_missing", "quantity_missing",
]
for col in flag_cols:
    df_panel[col] = df_panel[col].fillna(0).astype(int)

# 연/월 컬럼 다시 생성
df_panel["year"] = df_panel["date"].dt.year
df_panel["month"] = df_panel["date"].dt.month

print("최종 패널 예시:")
print(df_panel.head())

# 중복 체크 (item_id, date 조합이 유일한지)
print("중복 row 개수:", df_panel.duplicated(["item_id", "date"]).sum())


월별 집계 결과 예시:
    item_id   hs4       date    value    weight  quantity  value_zero  \
0  AANGBULD  4810 2022-01-01  14276.0   17625.0       0.0           1   
1  AANGBULD  4810 2022-02-01  52347.0   67983.0       0.0           0   
2  AANGBULD  4810 2022-03-01  53549.0   69544.0       0.0           1   
3  AANGBULD  4810 2022-05-01  26997.0   34173.0       0.0           1   
4  AANGBULD  4810 2022-06-01  84489.0  103666.0       0.0           1   

   weight_zero  quantity_zero  value_missing  weight_missing  \
0            1              1              0               0   
1            0              1              0               0   
2            1              1              0               0   
3            1              1              0               0   
4            1              1              0               0   

   quantity_missing  has_tx  
0                 0       1  
1                 0       1  
2                 0       1  
3                 0       1  
4            

In [7]:
df_panel.duplicated(["item_id", "date"]).sum()

np.int64(0)

In [8]:
df_panel["hs2"] = df_panel["hs4"].astype(str).str[:2]
df_panel["hs2_name_kr"] = df_panel["hs2"].map(hs2_map)

In [9]:
#df_panel.isnull().sum()
#df_panel[df_panel['seq'].isnull()]

#df_panel.duplicated().sum()
df_panel.head()


,item_id,date,hs4,value,weight,quantity,value_zero,weight_zero,quantity_zero,value_missing,weight_missing,quantity_missing,has_tx,year,month,hs2,hs2_name_kr
0,AANGBULD,2022-01-01,4810.0,14276.0,17625.0,0.0,1,1,1,0,0,0,1,2022,1,48,지·판지
1,AANGBULD,2022-02-01,4810.0,52347.0,67983.0,0.0,0,0,1,0,0,0,1,2022,2,48,지·판지
2,AANGBULD,2022-03-01,4810.0,53549.0,69544.0,0.0,1,1,1,0,0,0,1,2022,3,48,지·판지
3,AANGBULD,2022-04-01,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,2022,4,na,NaN
4,AANGBULD,2022-05-01,4810.0,26997.0,34173.0,0.0,1,1,1,0,0,0,1,2022,5,48,지·판지


item_id × month 형태의 시계열 매트릭스를 만들어야 함.

In [10]:
value_ts = df_panel.pivot(index='date', columns='item_id', values='value')
value_ts

item_id,AANGBULD,AHMDUILJ,ANWUJOKX,APQGTRMF,ATLDMDBO,AXULOHBQ,BEZYMBBT,BJALXPFS,BLANHGYY,BSRMSVTC,...,XIFHSOWQ,XIIEJNEE,XIPPENFQ,XMKRPGLB,XUOIQPFL,YSYHGLQK,ZCELVYQU,ZGJXVMNI,ZKENOUDA,ZXERAXWP
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,14276.0,242705.0,0.0,383999.0,143097177.0,11429.0,6034752.0,352532.0,0.0,1216269.0,...,7718.0,0.0,108699.0,879207.0,10871069.0,0.0,373859.0,1154724.0,122627.0,10867.0
2022-02-01,52347.0,120847.0,0.0,512813.0,103568323.0,16365.0,6221091.0,218947.0,891.0,156005.0,...,24829.0,81.0,78853.0,1650028.0,9573079.0,543.0,59900.0,1337622.0,38162.0,675.0
2022-03-01,53549.0,197317.0,0.0,217064.0,118403737.0,47307.0,4611537.0,331472.0,0.0,801085.0,...,102019.0,3198210.0,172237.0,1831614.0,8774783.0,766.0,31158.0,1662893.0,3592.0,20548.0
2022-04-01,0.0,126142.0,63580.0,470398.0,121873741.0,59524.0,5202039.0,17480.0,0.0,365605.0,...,33215.0,0.0,89681.0,705567.0,4713153.0,1108.0,594407.0,1561647.0,5613.0,662.0
2022-05-01,26997.0,71730.0,81670.0,539873.0,115024617.0,74828.0,6889516.0,234330.0,23141.0,168725.0,...,82348.0,81175.0,81387.0,1718634.0,4612292.0,859.0,648232.0,1603223.0,33924.0,18949.0
2022-06-01,84489.0,149138.0,26424.0,582317.0,65716075.0,36182.0,5338460.0,51692.0,13258.0,948322.0,...,42138.0,144.0,164749.0,1884328.0,3603001.0,1426.0,496737.0,1641942.0,74939.0,4566.0
2022-07-01,0.0,186617.0,8470.0,759980.0,146216818.0,70404.0,6549792.0,164195.0,12467.0,276971.0,...,87886.0,0.0,166570.0,385689.0,6383064.0,2413.0,210179.0,1815161.0,93976.0,12358.0
2022-08-01,0.0,169995.0,0.0,216019.0,97552978.0,156481.0,4476319.0,59646.0,26514.0,159911.0,...,38781.0,0.0,129194.0,766922.0,5530462.0,638.0,0.0,1546959.0,83364.0,5108.0
2022-09-01,0.0,140547.0,0.0,537693.0,72341427.0,231421.0,5714072.0,201135.0,26512.0,547603.0,...,16369.0,650.0,127122.0,654225.0,4808943.0,0.0,70748.0,1536799.0,53842.0,7954.0
